## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r'C:\Users\maryc\Documents\Class\World_Weather_Analysis\Weather_Database\cities.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,66.72,48,75,19.57,broken clouds
1,1,Kaitangata,NZ,-46.2817,169.8464,33.51,89,9,3.42,clear sky
2,2,Lobito,AO,-12.3481,13.5456,70.32,89,100,2.15,overcast clouds
3,3,Cabo San Lucas,MX,22.8909,-109.9124,81.70,81,100,0.00,overcast clouds
4,4,Jamestown,US,42.0970,-79.2353,84.51,50,100,13.27,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>= min_temp)]
preferred_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Cabo San Lucas,MX,22.8909,-109.9124,81.70,81,100,0.00,overcast clouds
4,4,Jamestown,US,42.0970,-79.2353,84.51,50,100,13.27,overcast clouds
6,6,Arraial Do Cabo,BR,-22.9661,-42.0278,82.36,65,0,14.97,clear sky
12,12,Bathsheba,BB,13.2167,-59.5167,88.41,79,40,14.97,scattered clouds
13,13,Les Cayes,HT,18.2000,-73.7500,84.94,70,78,7.72,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 3 to 682
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              129 non-null    int64  
 1   City                 129 non-null    object 
 2   Country              129 non-null    object 
 3   Lat                  129 non-null    float64
 4   Lng                  129 non-null    float64
 5   Max Temp             129 non-null    float64
 6   Humidity             129 non-null    int64  
 7   Cloudiness           129 non-null    int64  
 8   Wind Speed           129 non-null    float64
 9   Current Description  129 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 11.1+ KB


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
cities=preferred_cities.dropna()
cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 3 to 682
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              129 non-null    int64  
 1   City                 129 non-null    object 
 2   Country              129 non-null    object 
 3   Lat                  129 non-null    float64
 4   Lng                  129 non-null    float64
 5   Max Temp             129 non-null    float64
 6   Humidity             129 non-null    int64  
 7   Cloudiness           129 non-null    int64  
 8   Wind Speed           129 non-null    float64
 9   Current Description  129 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 11.1+ KB


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cabo San Lucas,MX,81.70,overcast clouds,22.8909,-109.9124,
4,Jamestown,US,84.51,overcast clouds,42.0970,-79.2353,
6,Arraial Do Cabo,BR,82.36,clear sky,-22.9661,-42.0278,
12,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,
13,Les Cayes,HT,84.94,broken clouds,18.2000,-73.7500,
14,Sao Joao Do Paraiso,BR,81.99,few clouds,-15.3136,-42.0144,
15,Kahului,US,80.62,scattered clouds,20.8947,-156.4700,
19,Butaritari,KI,81.93,scattered clouds,3.0707,172.7902,
21,Manono,CD,81.52,scattered clouds,-7.3000,27.4167,
27,Kavieng,PG,81.99,clear sky,-2.5744,150.7967,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"]=hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found....skipping.")
hotel_df.head()

Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.
Hotel not found....skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Cabo San Lucas,MX,81.70,overcast clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
4,Jamestown,US,84.51,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Arraial Do Cabo,BR,82.36,clear sky,-22.9661,-42.0278,Pousada Porto Praia
12,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,Atlantis Hotel
13,Les Cayes,HT,84.94,broken clouds,18.2000,-73.7500,Villa Orphee


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotels_results = hotel_df.dropna(how="any")
hotels_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 3 to 682
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 129 non-null    object 
 1   Country              129 non-null    object 
 2   Max Temp             129 non-null    float64
 3   Current Description  129 non-null    object 
 4   Lat                  129 non-null    float64
 5   Lng                  129 non-null    float64
 6   Hotel Name           129 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.1+ KB


In [25]:
# 8a. Create the output File (CSV)
output_data_file="hotel_cities.csv"
# 8b. Export the City_Data into a csv
hotels_results.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotels_results.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotels_results[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
# 11b. Display the figure
fig=gmaps.figure()
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))